In [1]:
import os 

os.environ['PYTHONPATH']

'.;C:\\Users\\Playdata\\ObjectDetection\\TF_oda2\\models;C:\\Users\\Playdata\\ObjectDetection\\TF_oda2\\models\\research'

In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from object_detection.utils import label_map_util, config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [3]:
PIPELINE_CONFIG_PATH = os.path.join('model', 'pipeline.config')
CHECK_POINT_PATH = os.path.join('model','checkpoint', 'ckpt-21')
LABEL_MAP_FILE_PATH = os.path.join('labelmap','label_map.pbtxt')

In [4]:
#pipline.config에 맞춰서 추출한 모델을 바탕으로 모델을 생성

# pipeline.config를 조회 // 컨피그 읽어옴.. 
config = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_PATH)

# pipeline.config의 model설정 정보를 넣어서 모델 생성
detection_model = model_builder.build(model_config= config['model'], is_training =False)

# detection_model -> 학습시킨 모델
# 모델에 학습시킨 checkpoint(weight)를 주입
# checkpoint 조회
ckpt = tf.compat.v2.train.Checkpoint(model = detection_model)
ckpt.restore(CHECK_POINT_PATH).expect_partial()

In [5]:
# detection을 실행하는 함수

# 순전파 처리 함수에 @tf.function decorator를 선언하면 실행 속도가 빨라진다.
@tf.function
def detect_func(image):
    """
    매개변수로 object detection을 수행할 대상 image(Tensor)를 받아서 detection처리.
    1. preprocessing(전처리): resize, normalization 작업
    2. detection(inference - 추론)
    3. detection결과를 post processing : Non Maximum Suppression
    4. post processing한 결과를 반환.
    """

    # 1. preprocessing
    image, shapes = detection_model.preprocess(image)
    # 2. 추론
    predict_dict = detection_model.predict(image, shapes)
    # 3. post processing
    result =detection_model.postprocess(predict_dict, shapes)

    return result # 4교시 수업.. 다시.. 

In [6]:
category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_FILE_PATH)
print(type(category_index))
category_index

<class 'dict'>


{1: {'id': 1, 'name': 'one'},
 2: {'id': 2, 'name': 'two'},
 3: {'id': 3, 'name': 'three'},
 4: {'id': 4, 'name': 'four'},
 5: {'id': 5, 'name': 'five'}}

In [7]:
# 맥에서는 터미널에서 export PYTHONPATH=$PYTHONPATH:[path]:[path]/slim 입력하고 주피터키시면 되세요!!
# path는 경로  직접 입력해주셔야하구요!


In [8]:
# 웹캠으로 부터 이미지를 받아서 추론한 결과를 화면에 보여주기.
cap = cv2.VideoCapture(0)
# 웹켐 width, height 조회
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [10]:
while True:
    ret, frame = cap.read() # 한 frame 읽기.
    frame = cv2.flip(frame,1) # 좌우 반전
    
    # BGR -> RGB (모델이 학습할때 RGB 모드로 학습했기 때문에 같은 형식으로 변환)

    image_np = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # 0 번축을 늘린다. Tensor로 변환
    input_tensor = tf.convert_to_tensor(image_np[np.newaxis,...], dtype = tf.float32)
    
    #추론
    post_detection = detect_func(input_tensor) #전처리 -> 추론 -> 후처리
    
    num_detections = int(post_detection.pop('num_detections'))
    # 추론한 결과들을 num_detection 개수 (detection한 물체의 개수) 만큼의 값만 남긴다.
    detections ={key:value[0, :num_detections].numpy()   for key, value in post_detection.items()}

    # 새로 구성한 결과 dictionary(detections)에 num_detections 값을 추가.
    detections['num_detections'] = num_detections
    # detection_classes는 검출한 box의 class 값을 label encording 된 값으로 가진다. float32로 반환되는 것dmf int로 변환 처리
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    
    
    
    MIN_CONF_THRESH = 0.7 # 물체가 있을 Confidence score가 0.5이상인 bounding box만 나오도록 하겠다.
    image_np_with_detection = image_np.copy() # detection한 원본 이미지의 카피본을 생성.
    img = viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detection, # 추론한 원본이미지
                    detections['detection_boxes'],  # bounding box 좌표
                    detections['detection_classes'] + 1, # bounding box내의 물체 index (class 확률에서 0은 첫번째 label,labelmap의 id는 1 부터 시작하기 때문에 +1)
                    detections['detection_scores'], #bounding box 내에 물체가 있을 확률(confidence score)
                    category_index,
                    use_normalized_coordinates = True, #bounding box의 좌표들이 normalize 되었는지 여부
                    max_boxes_to_draw = 200,
                    min_score_thresh = MIN_CONF_THRESH) # Confidence score가 얼마 이상인 bounding box만 나오도록 하겠다.
    
    
    # 결과 image를 RGB ->BGR 
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    # 화면에 출력
    cv2.imshow('frame', img)
    if cv2.waitKey(1) >0 : # 아무키나 입력하면 
        break
    
    
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-dn5w5exm\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


# post processing 결과 확인

In [11]:
img = cv2.imread('five.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = img[np.newaxis, ...]
input_tensor = tf.convert_to_tensor(img, dtype= tf.float32)

In [12]:
post_detection = detect_func(input_tensor)

In [13]:
post_detection.keys()

dict_keys(['detection_boxes', 'detection_scores', 'detection_classes', 'num_detections', 'raw_detection_boxes', 'raw_detection_scores', 'detection_multiclass_scores', 'detection_anchor_indices'])

In [14]:
post_detection['num_detections'].numpy() # Tensor -> ndarray
#num_detection : 후처리로 최종 결과로 나온 bounding box의 개수. 
#전체 bounding box에서 confidence score 순으로 내림 차순 한 뒤 NMS를 거쳐서 최종적으로 남은 bounding box의개수

array([100.], dtype=float32)

In [15]:
post_detection['detection_boxes'].shape
# [1, 100, 4] -> [추론한 이미지개수, num_detections, 좌표(x,y,w,h)]

TensorShape([1, 100, 4])

In [16]:
# bbox에 물체가 있을 확률 = confidence score
post_detection['detection_scores'].shape
#[1, 100] -> [추로한 이미지개수, num_detections]

TensorShape([1, 100])

In [17]:
post_detection['detection_boxes'][0, :10]  # 마지막 교시...

<tf.Tensor: shape=(10, 4), dtype=float32, numpy=
array([[0.585899  , 0.33339408, 0.9933591 , 0.632707  ],
       [0.67153436, 0.67636776, 0.9882881 , 0.90353   ],
       [0.        , 0.12148029, 1.        , 1.        ],
       [0.5910687 , 0.34480077, 0.9933917 , 0.6449523 ],
       [0.        , 0.18655479, 0.3199943 , 0.5079384 ],
       [0.        , 0.        , 0.3835048 , 0.20605263],
       [0.10778086, 0.        , 0.4977411 , 0.32233542],
       [0.67153436, 0.67636776, 0.9882881 , 0.90353   ],
       [0.5972187 , 0.        , 0.9824689 , 0.1816591 ],
       [0.83610964, 0.7273355 , 0.9993197 , 0.9454033 ]], dtype=float32)>

In [18]:
post_detection['detection_boxes'][0, 0]

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.585899  , 0.33339408, 0.9933591 , 0.632707  ], dtype=float32)>

In [19]:
post_detection['detection_scores']

<tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[0.9725988 , 0.6281108 , 0.35057196, 0.2724839 , 0.15958795,
        0.14284   , 0.08882222, 0.08877608, 0.07228383, 0.07189092,
        0.06729928, 0.0644708 , 0.05527753, 0.0516035 , 0.04836428,
        0.04740807, 0.04544836, 0.0445638 , 0.04317635, 0.04245842,
        0.04229394, 0.04067498, 0.03857976, 0.03775012, 0.03633496,
        0.03505528, 0.03480923, 0.03420192, 0.03180853, 0.03121236,
        0.03024733, 0.03023371, 0.02901089, 0.02875397, 0.02818114,
        0.02759275, 0.02748793, 0.02691084, 0.02684328, 0.02584034,
        0.02540469, 0.02519199, 0.02516478, 0.02473003, 0.02425498,
        0.02398628, 0.02325255, 0.02275136, 0.02270487, 0.02239472,
        0.02202275, 0.02178961, 0.02140054, 0.0210278 , 0.02087599,
        0.02059463, 0.02048776, 0.0202468 , 0.01960331, 0.01910436,
        0.01900312, 0.0189757 , 0.01892054, 0.01891473, 0.01875782,
        0.01854795, 0.01837516, 0.01769209, 0.01762491, 0.01759472

In [39]:
# 각 class 별 확률
post_detection['detection_multiclass_scores'].shape

TensorShape([1, 100, 6])

In [40]:
post_detection['detection_multiclass_scores'][0,0]

<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([3.3004880e-03, 4.2986274e-03, 6.1255395e-03, 1.9497991e-02,
       7.2723627e-04, 9.7259879e-01], dtype=float32)>